<a href="https://colab.research.google.com/github/annabellesauve/Empathy-Detection/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install transformers
! pip install tensorflow_addons

     |████████████████████████████████| 3.1 MB 7.0 MB/s 
     |████████████████████████████████| 596 kB 55.7 MB/s 
     |████████████████████████████████| 895 kB 47.6 MB/s 
     |████████████████████████████████| 59 kB 6.4 MB/s 
     |████████████████████████████████| 3.3 MB 22.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.1 MB 8.3 MB/s 


In [ ]:
# need if running locally
# import os
# os.add_dll_directory("C:/Program Files/NVIDIA GPU Computing Toolkit/CUDA/v11.5/bin")

In [ ]:
# paste this code in dev tool console to avoid disconnecting from the colab runtime
# function ClickConnect(){
# console.log("Working"); 
# document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
# }
# setInterval(ClickConnect,60000)

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
from transformers import RobertaTokenizer, TFRobertaModel, RobertaForMaskedLM, RobertaConfig, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import pandas as pd
import numpy as np
import math
import tqdm
import math
import os
from sklearn.model_selection import train_test_split
from keras import backend as K

# Data Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# load datasets directly from our repo

df_ER = pd.read_csv('https://raw.githubusercontent.com/annabellesauve/Empathy-Detection/main/Datasets/processed_emotionalreactions.csv')
ER_train, ER_test = train_test_split(df_ER, test_size=0.2)

df_EX = pd.read_csv('https://raw.githubusercontent.com/annabellesauve/Empathy-Detection/main/Datasets/processed_explorations-reddit.csv')
EX_train, EX_test = train_test_split(df_EX, test_size=0.2)

df_IN = pd.read_csv('https://raw.githubusercontent.com/annabellesauve/Empathy-Detection/main/Datasets/processed_interpretations-reddit.csv')
IN_train, IN_test = train_test_split(df_IN, test_size=0.2)

# Domain Adaptive Pre-Training

In [ ]:
# model_folder = '/content/drive/My Drive/empathy/pre_training_model'
# tokenizer_folder = '/content/drive/My Drive/empathy/tokenizer'

In [ ]:
# df_pretrain = pd.read_csv('https://raw.githubusercontent.com/annabellesauve/Empathy-Detection/main/Datasets/processed_emotionalreactions.csv', header=0, usecols=[1, 2])
# pretrain_train, pretrain_test = train_test_split(df_pretrain, test_size=0.2)

In [ ]:
# PRETRAIN_BATCH = 8
# PRETRAIN_EPOCHS = 3
# MAX_LEN = 128

# config = RobertaConfig(
#     vocab_size=8192,
#     max_position_embeddings=514,
#     num_attention_heads=12,
#     num_hidden_layers=6,
#     type_vocab_size=1,
# )

# robertabase_maskedLM_model = RobertaForMaskedLM(config=config)
# robertabase_tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=True)

In [ ]:
# from torch.utils.data.dataset import Dataset
# import torch

# class CustomDS(Dataset):
#   def __init__(self, df):
#     self.examples = []
        
#     for example in df.values:
#         x = robertabase_tokenizer.encode_plus(example, max_length = MAX_LEN, truncation=True, padding=True)
#         self.examples += [x.input_ids]

#   def __len__(self):
#       return len(self.examples)

#   def __getitem__(self, i):
#       return torch.tensor(self.examples[i])

In [ ]:
# pretrain_train_DS = CustomDS(pretrain_train['response_post'])
# pretrain_eval_DS = CustomDS(pretrain_test['response_post'])

In [ ]:
# data_collator = DataCollatorForLanguageModeling(
#     tokenizer=robertabase_tokenizer, mlm=True, mlm_probability=0.15
# )

# # Define the training arguments
# training_args = TrainingArguments(
#     output_dir=model_folder,
#     overwrite_output_dir=True,
#     evaluation_strategy = 'epoch',
#     num_train_epochs=PRETRAIN_EPOCHS,
#     per_device_train_batch_size=PRETRAIN_BATCH,
#     save_steps=8192,
#     #eval_steps=4096,
#     save_total_limit=1,
# )
# # Create the trainer for our model
# trainer = Trainer(
#     model=robertabase_maskedLM_model,
#     args=training_args,
#     data_collator=data_collator,
#     train_dataset=pretrain_train_DS,
#     eval_dataset=pretrain_eval_DS,
#     #prediction_loss_only=True,
# )

In [ ]:
# trainer.train()

In [ ]:
# trainer.save_model(model_folder)

# Encoders

In [ ]:
# robertabase model initialization, from huggingface
robertabase_tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=True)
robertabase_model = TFRobertaModel.from_pretrained('roberta-base')


# robertabase_model = RobertaForMaskedLM.from_pretrained(model_folder) # trying to load model that was pretrained - does not work since it's a masked language task model and the task in the encoder is not a masked language modeling task.


Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/627M [00:00<?, ?B/s]

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Roberta documentation: https://huggingface.co/transformers/model_doc/roberta.html?highlight=roberta#tfrobertamodel

**S Encoder**

In [ ]:
class S_Encoder(tf.Module):
  def __init__(self):
    super().__init__()

  def encode(self, seeker_post):
    seeker_post = seeker_post[:512]
    encoded_input = robertabase_tokenizer(seeker_post, return_tensors='tf')
    return robertabase_model(encoded_input)

**R Encoder**

In [ ]:
class R_Encoder(tf.Module):
  def __init__(self):
    super().__init__()

  def encode(self, response_post):
    response_post = response_post[:512]
    encoded_input = robertabase_tokenizer(response_post, return_tensors='tf')
    return robertabase_model(encoded_input)

# Attention Layer

In [ ]:
# single head attention layer
class AttentionLayer(tf.keras.Model):
  
  def __init__(self):
    super(AttentionLayer, self).__init__()

  def call(self, e_R, e_S, d):
    a = tf.math.multiply(tf.nn.softmax(tf.math.divide(tf.math.multiply(e_R, e_S), math.sqrt(d))), e_R)
    return a

# Model

In [ ]:
# add custom metrics
loss_tracker = tf.keras.metrics.Mean(name="loss")
empathy_metric = tf.keras.metrics.Mean(name="empathy loss")
rationale_metric = tf.keras.metrics.Mean(name="rationale loss")
empathy_accuracy = tf.keras.metrics.Accuracy(name="empathy accuracy")
rationale_accuracy = tf.keras.metrics.Accuracy(name="rationale accuracy")
f1_rationale = tf.keras.metrics.Mean(name="rationale f1")

# define f1 function for rationale 
def f1(y_true, y_pred):  
  tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1))) # true positive
  pos = K.sum(K.round(K.clip(y_true, 0, 1))) # possible positive
  pred = K.sum(K.round(K.clip(y_pred, 0, 1)))  # predicted positive

  pos = tf.add(tf.cast(pos, tf.float32), K.epsilon())
  pred = tf.add(tf.cast(pred, tf.float32), K.epsilon())
  
  recall = tf.cast(tp, tf.float32) / (pos)
  precision = tf.cast(tp, tf.float32) / (pred)

  return 2*((precision*recall)/(precision+recall+K.epsilon()))

# modeled after BiEncoderAttentionWithRationaleClassification() in original models.py
class Model(tf.keras.Model):

  def __init__(self, hidden_dropout_prob=0.2, rationale_num_labels=2, empathy_num_labels= 3, hidden_size=768):
    super(Model, self).__init__()

    self.hidden_dropout_prob = hidden_dropout_prob
    self.rationale_classifier = tf.keras.layers.Dense(rationale_num_labels, activation=None) # linear layer
    self.attention = AttentionLayer()
    self.rationale_num_labels = rationale_num_labels
    self.empathy_num_labels = empathy_num_labels
    self.empathy_classifier = EmpathyClassifier() # linear layer
    self.r_encoder = R_Encoder()
    self.s_encoder = S_Encoder()
    self.hidden_size = hidden_size
    self.lambda_EI = 1 
    self.lambda_RE = 0.1
  
  def call(self, inputs):
    input_ids_SP = inputs[0] # get seeker id 
    attention_mask_SP = inputs[1] # get seeker attention 
    input_ids_RP = inputs[2] # get response id 
    attention_mask_RP = inputs[3] # get response attention
    SP = inputs[4] # get seeker post
    RP = inputs[5] # get response post

    output_RP = []
    
    # convert to numpy array so we can get the correct string format
    SP = SP.numpy()
    RP = RP.numpy()

    # encode with s/r encoders, pass through attention layer
    for i in range(len(SP)):
      sp = self.s_encoder.encode(SP[i].decode("utf-8"))
      rp = self.r_encoder.encode(RP[i].decode("utf-8"))
      attn = self.attention(rp.pooler_output, sp.pooler_output, self.hidden_size)
      output_RP.append(rp.pooler_output + tf.nn.dropout(attn, self.hidden_dropout_prob))

    output_RP_tensor = tf.convert_to_tensor(output_RP, dtype=tf.float32)

    # empathy classification
    logits_empathy = self.empathy_classifier(output_RP_tensor)

    # dropout layer
    sequence_output = tf.nn.dropout(output_RP_tensor, self.hidden_dropout_prob)

    # rationale classification
    logits_rationale = self.rationale_classifier(sequence_output)

    return (logits_empathy, logits_rationale) # outputs
  
  def train_step(self, data): # override model.fit

    x, y = data

    e_id_SP = x[0] # get seeker id 
    e_attention_SP = x[1] # get seeker attention 
    e_id_RP = x[2] # get response id 
    e_attention_RP = x[3] # get response attention
    e_sp = x[4] # get seeker post
    e_rp = x[5] # get response post
    e_labels = y[0] # get labels
    e_rationales = y[1] # get rationales

    # get loss + gradients
    with tf.GradientTape() as tape:
        logits_empathy, logits_rationale = self(x)
        loss, loss_empathy, loss_rationale = overall_loss(e_labels, logits_empathy, e_rationales, logits_rationale, e_attention_RP, self.lambda_EI, self.lambda_RE)
    
    # compute gradients
    vars = self.trainable_variables
    gradients = tape.gradient(loss, vars)

    # update weights 
    self.optimizer.apply_gradients(zip(gradients, vars))

    # compute our own metrics + update them
    loss_tracker.update_state(loss)
    empathy_metric.update_state(loss_empathy)
    rationale_metric.update_state(loss_rationale)

    empathy_accuracy.update_state(e_labels, tf.argmax(tf.reshape(logits_empathy, [-1, 3]), 1))

    e_rationales = tf.reduce_mean(e_rationales, 1)

    rationale_accuracy.update_state(tf.reshape(e_rationales, [-1]), tf.argmax(tf.reshape(logits_rationale, [-1, 2]), 1))

    f1_score = f1(tf.reshape(e_rationales, [-1]), tf.argmax(tf.reshape(logits_rationale, [-1, 2]), 1))
    f1_rationale.update_state(f1_score)

    return {"loss": loss_tracker.result(), "empathy loss": empathy_metric.result(), "rationale loss": rationale_metric.result(), "empathy accuracy": empathy_accuracy.result(), "rationale accuracy": rationale_accuracy.result(), "rationale f1": f1_rationale.result()}
  
  def test_step(self, data): # override model.evaluate
    x, y = data

    e_id_SP = x[0] # get seeker id 
    e_attention_SP = x[1] # get seeker attention 
    e_id_RP = x[2] # get response id 
    e_attention_RP = x[3] # get response attention
    e_sp = x[4] # get seeker post
    e_rp = x[5] # get response post
    e_labels = y[0] # get labels
    e_rationales = y[1] # get rationales

    logits_empathy, logits_rationale = self(x)
    loss, loss_empathy, loss_rationale = overall_loss(e_labels, logits_empathy, e_rationales, logits_rationale, e_attention_RP, self.lambda_EI, self.lambda_RE)
    
    # compute our own metrics
    loss_tracker.update_state(loss)
    empathy_metric.update_state(loss_empathy)
    rationale_metric.update_state(loss_rationale)

    empathy_accuracy.update_state(e_labels, tf.argmax(tf.reshape(logits_empathy, [-1, 3]), 1))

    e_rationales = tf.reduce_mean(e_rationales, 1)
    rationale_accuracy.update_state(tf.reshape(e_rationales, [-1]), tf.argmax(tf.reshape(logits_rationale, [-1, 2]), 1))
    
    f1_score = f1(tf.reshape(e_rationales, [-1]), tf.argmax(tf.reshape(logits_rationale, [-1, 2]), 1))
    f1_rationale.update_state(f1_score)

    return {"loss": loss_tracker.result(), "empathy loss": empathy_metric.result(), "rationale loss": rationale_metric.result(), "empathy accuracy": empathy_accuracy.result(), "rationale accuracy": rationale_accuracy.result(), "rationale f1": f1_rationale.result()}
  
  @property 
  def metrics(self):
    return [loss_tracker, empathy_metric, rationale_metric, empathy_accuracy, rationale_accuracy, f1_rationale]




# Empathy Identification

In [ ]:
# modeled after RobertaClassificationHead in model.py
class EmpathyClassifier(tf.keras.Model):

  def __init__(self, hidden_dropout_prob=0.1, hidden_size=768, empathy_num_labels=3):
    super(EmpathyClassifier, self).__init__()

    self.dense = tf.keras.layers.Dense(hidden_size, activation=None)
    self.hidden_dropout_prob = hidden_dropout_prob
    self.out_proj = tf.keras.layers.Dense(empathy_num_labels, activation=None)
    self.relu = tf.keras.layers.ReLU()

  def call(self, features):
    x = features[:, :]
    x = tf.nn.dropout(x, self.hidden_dropout_prob)
    x = self.dense(x)
    x = self.relu(x)
    x = tf.nn.dropout(x, self.hidden_dropout_prob)
    x = self.out_proj(x)
    return x

# Loss functions

In [ ]:
# custom loss functions

# custom empathy loss with cross entropy
def empathy_loss(labels, logits):
  labels = tf.reshape(labels, [-1])
  logits = tf.reshape(logits, [-1, 3])
  loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)
  return loss

# custom rationale loss with cross entropy
def rationale_loss(labels, logits, attention):
  active_loss = tf.reshape(attention, [-1])
  logits = tf.reshape(logits, [-1, 2])

  if active_loss.shape[0] == 1:
    labels = tf.reshape(labels, [-1])
  else:
    labels = tf.reduce_mean(labels, 1)
    labels = tf.reshape(labels, [-1])

  loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)
  return loss

# custom overall loss
def overall_loss(labels_empathy, logits_empathy, labels_rationale, logits_rationale, attention, lambda_EI, lambda_RE):
  loss_empathy = empathy_loss(labels_empathy, logits_empathy)
  loss_rationale = rationale_loss(labels_rationale, logits_rationale, attention)

  loss = lambda_EI * loss_empathy + lambda_RE * loss_rationale
  return (loss, loss_empathy, loss_rationale)


# Initializing model

In [ ]:
# prepare the data
# tokenize inputs and attention mask, extract labels and rationales, create x and y to pass to model
def prepare(data):
  tokenizer_RP = robertabase_tokenizer.batch_encode_plus(data.response_post, add_special_tokens=True, max_length=64, padding='max_length', truncation=True)
  input_ids_RP = tokenizer_RP['input_ids']
  attention_masks_RP = tokenizer_RP['attention_mask']

  tokenizer_SP = robertabase_tokenizer.batch_encode_plus(data.seeker_post, add_special_tokens=True, max_length=64, padding='max_length', truncation=True)
  input_ids_SP = tokenizer_SP['input_ids']
  attention_masks_SP = tokenizer_RP['attention_mask']

  labels = np.array(data.level.values.astype(int))
  rationales = data.rationale_labels.values
  rationales = [list(map(int, i.split(','))) for i in rationales] # convert to int

  sp = data.seeker_post.values
  rp = data.response_post.values

  x = [tf.convert_to_tensor(input_ids_SP, dtype=tf.float32), tf.convert_to_tensor(attention_masks_SP, dtype=tf.float32), tf.convert_to_tensor(input_ids_RP, dtype=tf.float32), tf.convert_to_tensor(attention_masks_RP, dtype=tf.float32), sp, rp]
  y = [tf.convert_to_tensor(labels, dtype=tf.int64), tf.convert_to_tensor(rationales, dtype=tf.int64)]

  return x, y


In [ ]:
# tokenize input
print(ER_train.shape)
# ER
x_train_ER, y_train_ER = prepare(ER_train)
x_test_ER, y_test_ER = prepare(ER_test)

# IN 
x_train_IN, y_train_IN = prepare(IN_train)
x_test_IN, y_test_IN = prepare(IN_test)

# EX 
x_train_EX, y_train_EX = prepare(EX_train)
x_test_EX, y_test_EX = prepare(EX_test)

(2467, 7)


In [ ]:
# initialize the models

ER_model = Model()
IN_model = Model()
EX_model = Model()

optimizer_model = tfa.optimizers.AdamW(
    weight_decay=1e-2, # not given but default in py torch so probably used
    learning_rate=2e-5, # given
    epsilon=1e-8 # given
)

In [ ]:
tf.random.set_seed(12) # given seed

In [ ]:
checkpoint_path_ER = "/content/drive/My Drive/empathy/training_empathy_ER/cp.ckpt"
checkpoint_path_IN = "/content/drive/My Drive/empathy/training_empathy_IN/cp.ckpt"
checkpoint_path_EX = "/content/drive/My Drive/empathy/training_empathy_EX/cp.ckpt"
# checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback_ER = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path_ER,
                                                 save_weights_only=True,
                                                 verbose=1)

cp_callback_IN = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path_IN,
                                                 save_weights_only=True,
                                                 verbose=1)

cp_callback_EX = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path_EX,
                                                 save_weights_only=True,
                                                 verbose=1)

# ER

In [ ]:
# model training was successful - it has been saved
# the next blocks of code will use the saved model to run
# uncomment this code to train again


ER_model.compile(optimizer=optimizer_model, run_eagerly=True)

ER_model.fit(x=x_train_ER,
             y=y_train_ER,
             epochs=4,
             batch_size=32, 
             callbacks=[cp_callback_ER])


Epoch 1/4
78/78 [==============================] - ETA: 0s - loss: 0.9478 - empathy loss: 0.8948 - rationale loss: 0.5300 - empathy accuracy: 0.6356 - rationale accuracy: 0.9846 - rationale f1: 0.0000e+00 
Epoch 00001: saving model to /content/drive/My Drive/empathy/training_empathy_ER/cp.ckpt
78/78 [==============================] - 1922s 25s/step - loss: 0.9478 - empathy loss: 0.8948 - rationale loss: 0.5300 - empathy accuracy: 0.6356 - rationale accuracy: 0.9846 - rationale f1: 0.0000e+00
Epoch 2/4
78/78 [==============================] - ETA: 0s - loss: 0.9293 - empathy loss: 0.8801 - rationale loss: 0.4917 - empathy accuracy: 0.6619 - rationale accuracy: 1.0000 - rationale f1: 0.0000e+00 
Epoch 00002: saving model to /content/drive/My Drive/empathy/training_empathy_ER/cp.ckpt
78/78 [==============================] - 1906s 24s/step - loss: 0.9293 - empathy loss: 0.8801 - rationale loss: 0.4917 - empathy accuracy: 0.6619 - rationale accuracy: 1.0000 - rationale f1: 0.0000e+00
Epoch 

In [ ]:
ER_model_saved = Model()
ER_model_saved.load_weights(checkpoint_path_ER)
ER_model_saved.compile(optimizer=optimizer_model, run_eagerly=True) # recompile


# score = ER_model.evaluate(x=x_test_ER, 
                          # y=y_test_ER)

score = ER_model_saved.evaluate(x=x_test_ER,
                          y=y_test_ER)

print(f'Emotional reaction test scores\nOverall loss: {score[0]} \nEmpathy loss: {score[1]} \nRationale loss: {score[2]} \nEmpathy accuracy: {score[3]} \nRationale accuracy: {score[4]} \nRationale f1: {score[5]}')

20/20 [==============================] - 240s 12s/step - loss: 1.0361 - empathy loss: 0.9895 - rationale loss: 0.4658 - empathy accuracy: 0.6548 - rationale accuracy: 1.0000 - rationale f1: 0.0000e+00
Emotional reaction test scores
Overall loss: 1.0361191034317017 
Empathy loss: 0.9895352721214294 
Rationale loss: 0.4658363461494446 
Empathy accuracy: 0.6547812223434448 
Rationale accuracy: 1.0 
Rationale f1: 0.0


In [ ]:
# loading the model and training for 2 additional epochs (6 total)
ER_model_saved = Model()
ER_model_saved.load_weights(checkpoint_path_ER)
ER_model_saved.compile(optimizer=optimizer_model, run_eagerly=True) # recompile
ER_model.fit(x=x_train_ER,
             y=y_train_ER,
             initial_epoch=4,
             epochs=6,
             batch_size=32, 
             callbacks=[cp_callback_ER])

Epoch 5/6
78/78 [==============================] - ETA: 0s - loss: 1.0406 - empathy loss: 0.9940 - rationale loss: 0.4657 - empathy accuracy: 0.6619 - rationale accuracy: 1.0000 - rationale f1: 0.0000e+00 
Epoch 00005: saving model to /content/drive/My Drive/empathy/training_empathy_ER/cp.ckpt
78/78 [==============================] - 1900s 24s/step - loss: 1.0406 - empathy loss: 0.9940 - rationale loss: 0.4657 - empathy accuracy: 0.6619 - rationale accuracy: 1.0000 - rationale f1: 0.0000e+00
Epoch 6/6
78/78 [==============================] - ETA: 0s - loss: 1.0530 - empathy loss: 1.0067 - rationale loss: 0.4638 - empathy accuracy: 0.6619 - rationale accuracy: 1.0000 - rationale f1: 0.0000e+00 
Epoch 00006: saving model to /content/drive/My Drive/empathy/training_empathy_ER/cp.ckpt
78/78 [==============================] - 1900s 24s/step - loss: 1.0530 - empathy loss: 1.0067 - rationale loss: 0.4638 - empathy accuracy: 0.6619 - rationale accuracy: 1.0000 - rationale f1: 0.0000e+00


In [ ]:
ER_model_saved = Model()
ER_model_saved.load_weights(checkpoint_path_ER)
ER_model_saved.compile(optimizer=optimizer_model, run_eagerly=True) # recompile


# score = ER_model.evaluate(x=x_test_ER, 
                          # y=y_test_ER)

score = ER_model_saved.evaluate(x=x_test_ER,
                          y=y_test_ER)

print(f'Emotional reaction test scores\nOverall loss: {score[0]} \nEmpathy loss: {score[1]} \nRationale loss: {score[2]} \nEmpathy accuracy: {score[3]} \nRationale accuracy: {score[4]} \nRationale f1: {score[5]}')

20/20 [==============================] - 241s 12s/step - loss: 1.0589 - empathy loss: 1.0127 - rationale loss: 0.4621 - empathy accuracy: 0.6548 - rationale accuracy: 1.0000 - rationale f1: 0.0000e+00
Emotional reaction test scores
Overall loss: 1.0589051246643066 
Empathy loss: 1.0126922130584717 
Rationale loss: 0.46212995052337646 
Empathy accuracy: 0.6547812223434448 
Rationale accuracy: 1.0 
Rationale f1: 0.0


In [ ]:
# loading the model and training for 2 additional epochs (8 total)
ER_model_saved = Model()
ER_model_saved.load_weights(checkpoint_path_ER)
ER_model_saved.compile(optimizer=optimizer_model, run_eagerly=True) # recompile
ER_model.fit(x=x_train_ER,
             y=y_train_ER,
             initial_epoch=6,
             epochs=8,
             batch_size=32, 
             callbacks=[cp_callback_ER])

Epoch 7/8
78/78 [==============================] - ETA: 0s - loss: 1.0598 - empathy loss: 1.0136 - rationale loss: 0.4625 - empathy accuracy: 0.6619 - rationale accuracy: 1.0000 - rationale f1: 0.0000e+00 
Epoch 00007: saving model to /content/drive/My Drive/empathy/training_empathy_ER/cp.ckpt
78/78 [==============================] - 1875s 24s/step - loss: 1.0598 - empathy loss: 1.0136 - rationale loss: 0.4625 - empathy accuracy: 0.6619 - rationale accuracy: 1.0000 - rationale f1: 0.0000e+00
Epoch 8/8
26/78 [=========>....................] - ETA: 21:09 - loss: 1.0663 - empathy loss: 1.0201 - rationale loss: 0.4616 - empathy accuracy: 0.6394 - rationale accuracy: 1.0000 - rationale f1: 0.0000e+00

In [ ]:
ER_model_saved = Model()
ER_model_saved.load_weights(checkpoint_path_ER)
ER_model_saved.compile(optimizer=optimizer_model, run_eagerly=True) # recompile


# score = ER_model.evaluate(x=x_test_ER, 
                          # y=y_test_ER)

score = ER_model_saved.evaluate(x=x_test_ER,
                          y=y_test_ER)

print(f'Emotional reaction test scores\nOverall loss: {score[0]} \nEmpathy loss: {score[1]} \nRationale loss: {score[2]} \nEmpathy accuracy: {score[3]} \nRationale accuracy: {score[4]} \nRationale f1: {score[5]}')

20/20 [==============================] - 240s 12s/step - loss: 1.0632 - empathy loss: 1.0171 - rationale loss: 0.4603 - empathy accuracy: 0.6548 - rationale accuracy: 1.0000 - rationale f1: 0.0000e+00
Emotional reaction test scores
Overall loss: 1.0631636381149292 
Empathy loss: 1.0171290636062622 
Rationale loss: 0.46034476161003113 
Empathy accuracy: 0.6547812223434448 
Rationale accuracy: 1.0 
Rationale f1: 0.0


In [ ]:
predictions_empathy, predictions_rationale = ER_model_saved.predict(x=x_test_ER)

# print(predictions_empathy, predictions_rationale)


KeyboardInterrupt: ignored

In [ ]:
print(predictions_empathy.shape)
print(predictions_rationale.shape)

In [ ]:
predictions_empathy = predictions_empathy.reshape(617, 3) # reshape to get rid of useless [], value has to be correct based on the test set
predictions_rationale = predictions_rationale.reshape(617, 2)

In [ ]:
print(np.argmax(predictions_empathy, axis=1))
# print(predictions_empathy)

print(np.argmax(predictions_rationale, axis=1))
# print(predictions_rationale)

# IN

In [ ]:
# model training was successful - it has been saved
# the next blocks of code will use the saved model to run
# uncomment this code to train again

IN_model.compile(optimizer=optimizer_model, run_eagerly=True)

IN_model.fit(x=x_train_IN,
             y=y_train_IN,
             epochs=4,
             batch_size=32, 
             callbacks=[cp_callback_IN])

Epoch 1/4
78/78 [==============================] - ETA: 0s - loss: 0.9038 - empathy loss: 0.8393 - rationale loss: 0.6451 - empathy accuracy: 0.5347 - rationale accuracy: 0.6688 - rationale f1: 0.0000e+00 
Epoch 00001: saving model to /content/drive/My Drive/empathy/training_empathy_IN/cp.ckpt
78/78 [==============================] - 1863s 24s/step - loss: 0.9038 - empathy loss: 0.8393 - rationale loss: 0.6451 - empathy accuracy: 0.5347 - rationale accuracy: 0.6688 - rationale f1: 0.0000e+00
Epoch 2/4
78/78 [==============================] - ETA: 0s - loss: 0.8972 - empathy loss: 0.8572 - rationale loss: 0.3993 - empathy accuracy: 0.5391 - rationale accuracy: 1.0000 - rationale f1: 0.0000e+00 
Epoch 00002: saving model to /content/drive/My Drive/empathy/training_empathy_IN/cp.ckpt
78/78 [==============================] - 1875s 24s/step - loss: 0.8972 - empathy loss: 0.8572 - rationale loss: 0.3993 - empathy accuracy: 0.5391 - rationale accuracy: 1.0000 - rationale f1: 0.0000e+00
Epoch 

In [ ]:
IN_model_saved = Model()
IN_model_saved.load_weights(checkpoint_path_IN)


IN_model_saved.compile(optimizer=optimizer_model, run_eagerly=True) # recompile

score = IN_model_saved.evaluate(x=x_test_IN,
                          y=y_test_IN)

print(f'Interpretation test scores\nOverall loss: {score[0]} \nEmpathy loss: {score[1]} \nRationale loss: {score[2]} \nEmpathy accuracy: {score[3]} \nRationale accuracy: {score[4]} \nRationale f1: {score[5]}')

20/20 [==============================] - 244s 12s/step - loss: 0.9733 - empathy loss: 0.9349 - rationale loss: 0.3836 - empathy accuracy: 0.4797 - rationale accuracy: 1.0000 - rationale f1: 0.0000e+00
Interpretation test scores
Overall loss: 0.9732635617256165 
Empathy loss: 0.934906542301178 
Rationale loss: 0.3835689127445221 
Empathy accuracy: 0.4797406792640686 
Rationale accuracy: 1.0 
Rationale f1: 0.0


In [ ]:
# loading the model and training for 2 additional epochs (6 total)
IN_model_saved = Model()
IN_model_saved.load_weights(checkpoint_path_IN)
IN_model.compile(optimizer=optimizer_model, run_eagerly=True)

IN_model.fit(x=x_train_IN,
             y=y_train_IN,
             initial_epoch=4,
             epochs=6,
             batch_size=32, 
             callbacks=[cp_callback_IN])

Epoch 5/6
78/78 [==============================] - ETA: 0s - loss: 0.9702 - empathy loss: 0.9316 - rationale loss: 0.3868 - empathy accuracy: 0.5391 - rationale accuracy: 1.0000 - rationale f1: 0.0000e+00 
Epoch 00005: saving model to /content/drive/My Drive/empathy/training_empathy_IN/cp.ckpt
78/78 [==============================] - 1936s 25s/step - loss: 0.9702 - empathy loss: 0.9316 - rationale loss: 0.3868 - empathy accuracy: 0.5391 - rationale accuracy: 1.0000 - rationale f1: 0.0000e+00
Epoch 6/6
78/78 [==============================] - ETA: 0s - loss: 0.9835 - empathy loss: 0.9441 - rationale loss: 0.3939 - empathy accuracy: 0.5391 - rationale accuracy: 1.0000 - rationale f1: 0.0000e+00 
Epoch 00006: saving model to /content/drive/My Drive/empathy/training_empathy_IN/cp.ckpt
78/78 [==============================] - 1925s 25s/step - loss: 0.9835 - empathy loss: 0.9441 - rationale loss: 0.3939 - empathy accuracy: 0.5391 - rationale accuracy: 1.0000 - rationale f1: 0.0000e+00


In [ ]:
IN_model_saved = Model()
IN_model_saved.load_weights(checkpoint_path_IN)


IN_model_saved.compile(optimizer=optimizer_model, run_eagerly=True) # recompile

score = IN_model_saved.evaluate(x=x_test_IN,
                          y=y_test_IN)

print(f'Interpretation test scores\nOverall loss: {score[0]} \nEmpathy loss: {score[1]} \nRationale loss: {score[2]} \nEmpathy accuracy: {score[3]} \nRationale accuracy: {score[4]} \nRationale f1: {score[5]}')

20/20 [==============================] - 239s 12s/step - loss: 0.9958 - empathy loss: 0.9561 - rationale loss: 0.3975 - empathy accuracy: 0.4797 - rationale accuracy: 1.0000 - rationale f1: 0.0000e+00
Interpretation test scores
Overall loss: 0.9958151578903198 
Empathy loss: 0.9560658931732178 
Rationale loss: 0.3974931240081787 
Empathy accuracy: 0.4797406792640686 
Rationale accuracy: 1.0 
Rationale f1: 0.0


In [ ]:
# loading the model and training for 2 additional epochs (8 total)
IN_model_saved = Model()
IN_model_saved.load_weights(checkpoint_path_IN)
IN_model.compile(optimizer=optimizer_model, run_eagerly=True)

IN_model.fit(x=x_train_IN,
             y=y_train_IN,
             initial_epoch=6,
             epochs=8,
             batch_size=32, 
             callbacks=[cp_callback_IN])

Epoch 7/8
78/78 [==============================] - ETA: 0s - loss: 0.9851 - empathy loss: 0.9322 - rationale loss: 0.5292 - empathy accuracy: 0.5091 - rationale accuracy: 0.9854 - rationale f1: 0.0000e+00 
Epoch 00007: saving model to /content/drive/My Drive/empathy/training_empathy_IN/cp.ckpt
78/78 [==============================] - 1841s 24s/step - loss: 0.9851 - empathy loss: 0.9322 - rationale loss: 0.5292 - empathy accuracy: 0.5091 - rationale accuracy: 0.9854 - rationale f1: 0.0000e+00
Epoch 8/8
78/78 [==============================] - ETA: 0s - loss: 0.9767 - empathy loss: 0.9276 - rationale loss: 0.4913 - empathy accuracy: 0.5213 - rationale accuracy: 1.0000 - rationale f1: 0.0000e+00 
Epoch 00008: saving model to /content/drive/My Drive/empathy/training_empathy_IN/cp.ckpt
78/78 [==============================] - 1827s 23s/step - loss: 0.9767 - empathy loss: 0.9276 - rationale loss: 0.4913 - empathy accuracy: 0.5213 - rationale accuracy: 1.0000 - rationale f1: 0.0000e+00


In [ ]:
IN_model_saved = Model()
IN_model_saved.load_weights(checkpoint_path_IN)


IN_model_saved.compile(optimizer=optimizer_model, run_eagerly=True) # recompile

score = IN_model_saved.evaluate(x=x_test_IN,
                          y=y_test_IN)

print(f'Interpretation test scores\nOverall loss: {score[0]} \nEmpathy loss: {score[1]} \nRationale loss: {score[2]} \nEmpathy accuracy: {score[3]} \nRationale accuracy: {score[4]} \nRationale f1: {score[5]}')

20/20 [==============================] - 230s 11s/step - loss: 0.9997 - empathy loss: 0.9518 - rationale loss: 0.4795 - empathy accuracy: 0.5478 - rationale accuracy: 1.0000 - rationale f1: 0.0000e+00
Interpretation test scores
Overall loss: 0.9997437000274658 
Empathy loss: 0.9517909288406372 
Rationale loss: 0.4795263111591339 
Empathy accuracy: 0.5478119850158691 
Rationale accuracy: 1.0 
Rationale f1: 0.0


In [ ]:
predictions_empathy, predictions_rationale = IN_model_saved.predict(x=x_test_IN)

# print(predictions_empathy, predictions_rationale)


In [ ]:
# print(predictions_empathy.shape)
# print(predictions_rationale.shape)

predictions_empathy = predictions_empathy.reshape(617, 3)
predictions_rationale = predictions_rationale.reshape(617, 2)

In [ ]:
print(np.argmax(predictions_empathy, axis=1))
# print(predictions_empathy)

print(np.argmax(predictions_rationale, axis=1))
# print(predictions_rationale)

# EX


In [ ]:
# model training was successful - it has been saved
# the next blocks of code will use the saved model to run
# uncomment this code to train again

EX_model.compile(optimizer=optimizer_model, run_eagerly=True)

EX_model.fit(x=x_train_EX,
             y=y_train_EX,
             epochs=4,
             batch_size=32, 
             callbacks=[cp_callback_EX])

Epoch 1/4
78/78 [==============================] - ETA: 0s - loss: 0.6406 - empathy loss: 0.5885 - rationale loss: 0.5207 - empathy accuracy: 0.8448 - rationale accuracy: 0.9392 - rationale f1: 0.0000e+00 
Epoch 00001: saving model to /content/drive/My Drive/empathy/training_empathy_EX/cp.ckpt
78/78 [==============================] - 1829s 23s/step - loss: 0.6406 - empathy loss: 0.5885 - rationale loss: 0.5207 - empathy accuracy: 0.8448 - rationale accuracy: 0.9392 - rationale f1: 0.0000e+00
Epoch 2/4
78/78 [==============================] - ETA: 0s - loss: 0.6590 - empathy loss: 0.6168 - rationale loss: 0.4223 - empathy accuracy: 0.8456 - rationale accuracy: 1.0000 - rationale f1: 0.0000e+00 
Epoch 00002: saving model to /content/drive/My Drive/empathy/training_empathy_EX/cp.ckpt
78/78 [==============================] - 1862s 24s/step - loss: 0.6590 - empathy loss: 0.6168 - rationale loss: 0.4223 - empathy accuracy: 0.8456 - rationale accuracy: 1.0000 - rationale f1: 0.0000e+00
Epoch 

In [ ]:
EX_model_saved = Model()
EX_model_saved.load_weights(checkpoint_path_EX)


EX_model_saved.compile(optimizer=optimizer_model, run_eagerly=True) # recompile

score = EX_model_saved.evaluate(x=x_test_EX,
                          y=y_test_EX)

print(f'Explorations test scores\nOverall loss: {score[0]} \nEmpathy loss: {score[1]} \nRationale loss: {score[2]} \nEmpathy accuracy: {score[3]} \nRationale accuracy: {score[4]} \nRationale f1: {score[5]}')

20/20 [==============================] - 230s 11s/step - loss: 0.8302 - empathy loss: 0.7880 - rationale loss: 0.4214 - empathy accuracy: 0.8395 - rationale accuracy: 1.0000 - rationale f1: 0.0000e+00
Explorations test scores
Overall loss: 0.8301589488983154 
Empathy loss: 0.7880163192749023 
Rationale loss: 0.4214257299900055 
Empathy accuracy: 0.8395462036132812 
Rationale accuracy: 1.0 
Rationale f1: 0.0


In [ ]:
# load and train for 2 additional epochs (6 total)
EX_model_saved = Model()
EX_model_saved.load_weights(checkpoint_path_EX)

EX_model_saved.compile(optimizer=optimizer_model, run_eagerly=True) # recompile
EX_model_saved.fit(x=x_train_EX,
             y=y_train_EX,
             initial_epoch = 4,
             epochs=6,
             batch_size=32, 
             callbacks=[cp_callback_EX])

Epoch 5/6
78/78 [==============================] - ETA: 0s - loss: 0.8473 - empathy loss: 0.8048 - rationale loss: 0.4245 - empathy accuracy: 0.8472 - rationale accuracy: 1.0000 - rationale f1: 0.0000e+00 
Epoch 00005: saving model to /content/drive/My Drive/empathy/training_empathy_EX/cp.ckpt
78/78 [==============================] - 1754s 22s/step - loss: 0.8473 - empathy loss: 0.8048 - rationale loss: 0.4245 - empathy accuracy: 0.8472 - rationale accuracy: 1.0000 - rationale f1: 0.0000e+00
Epoch 6/6
78/78 [==============================] - ETA: 0s - loss: 0.8784 - empathy loss: 0.8355 - rationale loss: 0.4289 - empathy accuracy: 0.8472 - rationale accuracy: 1.0000 - rationale f1: 0.0000e+00 
Epoch 00006: saving model to /content/drive/My Drive/empathy/training_empathy_EX/cp.ckpt
78/78 [==============================] - 1741s 22s/step - loss: 0.8784 - empathy loss: 0.8355 - rationale loss: 0.4289 - empathy accuracy: 0.8472 - rationale accuracy: 1.0000 - rationale f1: 0.0000e+00


In [ ]:
EX_model_saved = Model()
EX_model_saved.load_weights(checkpoint_path_EX)


EX_model_saved.compile(optimizer=optimizer_model, run_eagerly=True) # recompile

score = EX_model_saved.evaluate(x=x_test_EX,
                          y=y_test_EX)

print(f'Explorations test scores\nOverall loss: {score[0]} \nEmpathy loss: {score[1]} \nRationale loss: {score[2]} \nEmpathy accuracy: {score[3]} \nRationale accuracy: {score[4]} \nRationale f1: {score[5]}')

20/20 [==============================] - 218s 11s/step - loss: 0.9000 - empathy loss: 0.8569 - rationale loss: 0.4306 - empathy accuracy: 0.8331 - rationale accuracy: 1.0000 - rationale f1: 0.0000e+00
Explorations test scores
Overall loss: 0.8999569416046143 
Empathy loss: 0.8568935394287109 
Rationale loss: 0.4306330978870392 
Empathy accuracy: 0.8330631852149963 
Rationale accuracy: 1.0 
Rationale f1: 0.0


In [ ]:
# load and train for 2 additional epochs (8 total)
EX_model_saved = Model()
EX_model_saved.load_weights(checkpoint_path_EX)

EX_model_saved.compile(optimizer=optimizer_model, run_eagerly=True) # recompile
EX_model_saved.fit(x=x_train_EX,
             y=y_train_EX,
             initial_epoch = 6,
             epochs=8,
             batch_size=32, 
             callbacks=[cp_callback_EX])

Epoch 7/8
78/78 [==============================] - ETA: 0s - loss: 0.9070 - empathy loss: 0.8637 - rationale loss: 0.4325 - empathy accuracy: 0.8472 - rationale accuracy: 1.0000 - rationale f1: 0.0000e+00 
Epoch 00007: saving model to /content/drive/My Drive/empathy/training_empathy_EX/cp.ckpt
78/78 [==============================] - 1726s 22s/step - loss: 0.9070 - empathy loss: 0.8637 - rationale loss: 0.4325 - empathy accuracy: 0.8472 - rationale accuracy: 1.0000 - rationale f1: 0.0000e+00
Epoch 8/8
78/78 [==============================] - ETA: 0s - loss: 0.9341 - empathy loss: 0.8906 - rationale loss: 0.4353 - empathy accuracy: 0.8472 - rationale accuracy: 1.0000 - rationale f1: 0.0000e+00 
Epoch 00008: saving model to /content/drive/My Drive/empathy/training_empathy_EX/cp.ckpt
78/78 [==============================] - 1752s 22s/step - loss: 0.9341 - empathy loss: 0.8906 - rationale loss: 0.4353 - empathy accuracy: 0.8472 - rationale accuracy: 1.0000 - rationale f1: 0.0000e+00


In [ ]:
EX_model_saved = Model()
EX_model_saved.load_weights(checkpoint_path_EX)


EX_model_saved.compile(optimizer=optimizer_model, run_eagerly=True) # recompile

score = EX_model_saved.evaluate(x=x_test_EX,
                          y=y_test_EX)

print(f'Explorations test scores\nOverall loss: {score[0]} \nEmpathy loss: {score[1]} \nRationale loss: {score[2]} \nEmpathy accuracy: {score[3]} \nRationale accuracy: {score[4]} \nRationale f1: {score[5]}')

20/20 [==============================] - 222s 11s/step - loss: 0.9516 - empathy loss: 0.9080 - rationale loss: 0.4366 - empathy accuracy: 0.8331 - rationale accuracy: 1.0000 - rationale f1: 0.0000e+00
Explorations test scores
Overall loss: 0.9516453146934509 
Empathy loss: 0.907990038394928 
Rationale loss: 0.43655163049697876 
Empathy accuracy: 0.8330631852149963 
Rationale accuracy: 1.0 
Rationale f1: 0.0


In [ ]:
predictions_empathy, predictions_rationale = EX_model_saved.predict(x=x_test_EX)

# print(predictions_empathy, predictions_rationale)


In [ ]:
# print(predictions_empathy.shape)
# print(predictions_rationale.shape)

predictions_empathy = predictions_empathy.reshape(617, 3)
predictions_rationale = predictions_rationale.reshape(617, 2)

In [ ]:
print(np.argmax(predictions_empathy, axis=1))
# print(predictions_empathy)

print(np.argmax(predictions_rationale, axis=1))
# print(predictions_rationale)